In [1]:
import sys, os 
import glob
from skimage import io
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
from torch.autograd import Variable

In [2]:
sys.path.append("../../deep_fit/")
sys.path.append("../../deep_fit/SFD_pytorch/")
# sys.path.append("../../deep_fit/utils/")

In [3]:
import utils.sfd as sfd
import fan.face_alignment as face_alignment
import image_preprocess

In [4]:
sfdnet = sfd.s3fd_create_net("../../deep_fit/weights/s3fd_convert.pth")
fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False, enable_cuda=True)

In [5]:
image_list = glob.glob(os.path.join('../datasets/ImageFolder/', "*.jpg"))

In [ ]:
batch_size = 16

In [ ]:
%%time
for i in tqdm(range(0, len(image_list), batch_size)):
    From = i
    To = min(i+batch_size, len(image_list))
    images_batch = []
    for img_path in image_list[From:To]:
        images_batch.append(io.imread(img_path))
    boxes_batch = sfd.s3fd_detect(sfdnet, images_batch)

    img_crops = None    ## crop된 이미지 배치
    
    for i in range(len(images_batch)):
        center, scale = image_preprocess.get_center_scale_from_rectangle(boxes_batch[i])
        img_crop = image_preprocess.crop(images_batch[i], center, scale, resolution=256)
        if img_crops is None:
            img_crops = img_crop.transpose(2, 0, 1)[np.newaxis, ]
        else:
            img_crops = np.concatenate([img_crops, img_crop.transpose(2,0,1)[np.newaxis, ]], axis=0)
            #         np.save(image_list[From + i] + "crop256", img_crop)

    img_torch  =torch.from_numpy(img_crops).float().div(255.0)
    img_var = Variable(img_torch, volatile=True).cuda()
    heatmap = fa.face_alignemnt_net(img_var)[-1].data.cpu().numpy()
    for i in range(len(images_batch)):
        pass
#         np.save(image_list[From+i] + "heatmap", heatmap[i])
    break

In [ ]:
inp = torch.from_numpy(img_crop.transpose((2, 0, 1))).float().div(255.0).unsqueeze_(0)
inp = Variable(inp, volatile=True).cuda()

In [ ]:
out = fa.face_alignemnt_net(inp)[-1].data.cpu()
io.imshow(img_crop)

In [ ]:
from PIL import ImageEnhance, Image
from skimage import io, transform

In [ ]:
import random

In [ ]:
def RandomEnhanceBrightness(img):
    # factor = random.uniform(0.5, 1.5)
    factor = random.gauss(1, 0.2)
    factor = np.clip(factor, 0.7, 1.3)
    return ImageEnhance.Brightness(img).enhance(factor)


def RandomEnhanceColor(img):
    factor = random.gauss(1, 0.2)
    factor = np.clip(factor, 0., 1.5)
    return ImageEnhance.Color(img).enhance(factor)


def RandomEnhanceContrast(img):
    factor = random.gauss(1, 0.2)
    factor = np.clip(factor, 0.8, 2)
    return ImageEnhance.Contrast(img).enhance(factor)


def RandomEnhanceSharpness(img):
    factor = random.gauss(1, 0.3)
    factor = np.clip(factor, -1, 5)
    return ImageEnhance.Sharpness(img).enhance(factor)


In [ ]:
import torchvision.transforms as transforms

In [ ]:
tr = transforms.Compose([
        Image.fromarray,
        RandomEnhanceBrightness,
        RandomEnhanceColor,
        RandomEnhanceContrast,
        RandomEnhanceSharpness])

In [ ]:
pimage = Image.fromarray(img_crop)
pimage = ImageEnhance.Brightness(pimage).enhance(0.5)
pimage = ImageEnhance.Color(pimage).enhance(1.5)
pimage = ImageEnhance.Contrast(pimage).enhance(1.2)
pimage = ImageEnhance.Sharpness(pimage).enhance(0.5)
pimage

In [ ]:
tr(img_crop)

In [ ]:
area = [0, 17, 22, 27, 31, 36, 42, 48, 60, 68]

In [ ]:
io.imshow(out[0].sum(dim=0).numpy())